In [0]:
# Cargar el CSV más reciente desde el volumen
df_nuevos = (
    spark.read.option("header", True)
              .option("inferSchema", True)
              .csv("/Volumes/workspace/default/retail_lakehouse/Online Retail Data Set.csv")
)

# Leer la tabla delta actual (bronze)
from delta.tables import DeltaTable
from pyspark.sql.functions import col

ruta_bronze = "/Volumes/workspace/default/retail_lakehouse/bronze"
delta_bronze = DeltaTable.forPath(spark, ruta_bronze)
df_bronze_actual = delta_bronze.toDF()

# Detectar registros nuevos (que no están en la capa Bronze)
df_nuevos_unicos = df_nuevos.alias("nuevos").join(
    df_bronze_actual.alias("existente"),
    on=[
        col("nuevos.InvoiceNo") == col("existente.InvoiceNo"),
        col("nuevos.StockCode") == col("existente.StockCode")
    ],
    how="left_anti"  # ← Devuelve solo los nuevos
)

# Mostrar cuántos nuevos se detectaron
print(f"🆕 Registros nuevos detectados: {df_nuevos_unicos.count()}")

# Insertar nuevos en la tabla Bronze
df_nuevos_unicos.write.format("delta").mode("append").save(ruta_bronze)

# Crear o actualizar vista para consultas
spark.read.format("delta").load(ruta_bronze).createOrReplaceTempView("bronze_online_retail")